# Imports

In [ ]:
# set root directory path
import os
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())
src_path = os.environ.get("PROJECT_SRC")
os.chdir(src_path)

src_path

In [ ]:
import pypsa
import matplotlib.pyplot as plt

plt.style.use("ggplot")

import pandas as pd

import data_reader_writer

# Generation capacities of the Clean Power scenarios

In [ ]:
further_flex: dict[str, float] = {
    "Wind Offshore": 50_600.0,
    "Wind Onshore": 27_300.0,
    "Solar Photovoltaics": 47_400.0,
    "CCS Biomass": 1_023.256,
    "Biomass (co-firing)": 537.6744,
    "Biomass (dedicated)": 2_439.0696,
    "Hydrogen": 150.0,
    "CCS Gas": 150.0,
    "Nuclear": 3_500.0,
    "Natural Gas": 35_000.0,
    "Large Hydro": 6_984.7407,
    "Small Hydro": 915.259,
    "Oil": 0.0,
}

new_dispatch: dict[str, float] = {
    "Wind Offshore": 43_100.0,
    "Wind Onshore": 27_300.0,
    "Solar Photovoltaics": 47_400.0,
    "CCS Biomass": 1_198.631,
    "Biomass (co-firing)": 629.847154,
    "Biomass (dedicated)": 2_857.195817,
    "Hydrogen": 1_350.0,
    "CCS Gas": 1_350.0,
    "Nuclear": 4_100.0,
    "Natural Gas": 35_000.0,
    "Large Hydro": 4_067.064205,
    "Small Hydro": 532.9356203,
    "Oil": 0.0,
}

# Set up simulation

##### 09/01/2025

Set up the inputs for the simulation.

Baseline year is used to determine which historical load profile and weather dataset is used for the future year that is modelled. National Grid ESO's FES modellers used 2012 as the baseline year.

In [ ]:
start: str   = "2030-01-01 00:00:00"
end: str     = "2030-12-31 23:30:00"

year: int = int(start[0:4])
time_step: float = 1.0  # time step as fraction of hour

baseline_year: int = 2012

save_capacity_figures: bool = True
save_timestep_figures: bool = True

In [ ]:
scenarios: list[str] = [
    "Leading The Way",
    "Consumer Transformation",
    "System Transformation",
    "Further Flex and Renewables",
    "New Dispatch",
]

figure_naming_list: list[str] = ["LTW", "CT", "ST", "FFR", "ND"]

counter = 0

##### 06/02/2025

Run `data_writer` to adjust the csv files depending on the scenario.

In [ ]:
scenario = "Leading The Way" if (counter == 3 or counter == 4) else "Consumer Transformation"

figure_name = figure_naming_list[counter]

data_reader_writer.data_writer(
    start,
    end,
    time_step,
    year,
    demand_dataset = "eload",
    year_baseline = baseline_year,
    scenario = scenario,
    FES = 2022,
    merge_generators = True,
    scale_to_peak = True,
    networkmodel = "Reduced",
    P2G = True,
)

##### 12/02/2025

Adjust the `generator.csv` under the `PyPSA/LOPF` folder to the new generation capacities.

In [ ]:
if counter == 3 or counter == 4:
    df_new_gen = pd.Series(further_flex if counter == 3 else new_dispatch)

    df_gen_p_nom = pd.read_csv("LOPF_data/generators.csv")
    p_nom_by_carrier = df_gen_p_nom.groupby("carrier")["p_nom"].sum()

    ratios = df_new_gen / p_nom_by_carrier
    ratios = ratios.fillna(0)

    adjust_csv_df: pd.DataFrame = pd.read_csv(
        "C:/Programming/PyPSA-GB_2030/PyPSA-GB/LOPF_data/generators.csv"
    )

    adjust_csv_df["p_nom"] = (adjust_csv_df["p_nom"]
                              * adjust_csv_df["carrier"].map(ratios))

    adjust_csv_df["p_nom"] = adjust_csv_df["p_nom"].fillna(0)

    adjust_csv_df.to_csv(
        "C:/Programming/PyPSA-GB_2030/PyPSA-GB/LOPF_data/generators.csv",
        index=False
    )

# Create network object for the optimisation

##### 12/02/2025

In [ ]:
network = pypsa.Network()
network.import_from_csv_folder("LOPF_data")
contingency_factor = 4
network.lines.s_max_pu *= contingency_factor

# Running the optimisation

##### 15/02/2025

Use `pypsa.optimize` to optimise the generation and storage units for the given scenario over the time period provided.

https://pypsa.readthedocs.io/en/stable/user-guide/optimal-power-flow.html

In [ ]:
network.optimize(
    solver_name = "gurobi",
    snapshots = network.snapshots,
)

# Graph the generation capacity by technology

##### 15/02/2025

In [ ]:
p_by_carrier = network.generators_t.p.groupby(
    network.generators.carrier,
    axis = 1
).sum()

storage_by_carrier = network.storage_units_t.p.groupby(
    network.storage_units.carrier,
    axis = 1
).sum()

# to show on graph set the negative storage values to zero
storage_by_carrier[storage_by_carrier < 0] = 0

p_by_carrier = pd.concat([p_by_carrier, storage_by_carrier], axis = 1)

# interconnector import
imp = network.links_t.p0.copy()
imp[imp < 0] = 0
imp["Interconnectors Import"] = imp.sum(axis = 1)
interconnector_import = imp[["Interconnectors Import"]]

p_by_carrier = pd.concat([p_by_carrier, interconnector_import], axis = 1)

# interconnector export
exp = network.links_t.p0.copy()
exp[exp > 0] = 0
exp['Interconnectors Export'] = exp.sum(axis=1)
interconnector_export = exp[['Interconnectors Export']]

# group biomass stuff
p_by_carrier['Biomass'] = (
    p_by_carrier['Biomass (dedicated)'] + p_by_carrier['Biomass (co-firing)']
)

# rename the hydro bit
p_by_carrier = p_by_carrier.rename(
    columns={'Large Hydro': 'Hydro'}
)
p_by_carrier = p_by_carrier.rename(
    columns={'Interconnector': 'Interconnectors Import'}
)

# group values by technology in the dataframe
generators_p_nom = network.generators.p_nom.groupby(
    network.generators.carrier
).sum().sort_values()

if year > 2020:
    generators_p_nom.drop('Unmet Load', inplace = True)
generators_p_nom.drop(generators_p_nom[generators_p_nom < 50].index, inplace = True)

# bar chart
plt.rcParams.update({'font.size': 12})
plt.figure(figsize=(10,4))
plt.bar(generators_p_nom.index, generators_p_nom.values / 1000)
plt.xticks(generators_p_nom.index, rotation=90)
plt.ylabel('GW')
plt.grid(color='grey', linewidth=1, axis='both', alpha=0.5)
plt.title(
    f"Installed capacity in year {year} for {scenarios[counter]}"
)

# save figures if bool is true
if save_capacity_figures:
    plt.savefig(
        f"path/to/project/Clean_Power_Analysis/Graphs/{figure_name}_Capacity.png",
        format="png",
        bbox_inches="tight",
    )
    plt.savefig(
        f"path/to/project/Clean_Power_Analysis/Graphs/{figure_name}_Capacity.pdf",
        format="pdf",
        bbox_inches="tight"
    )

plt.show()

# Graph the time step generation over the given time period

##### 15/02/2025

In [ ]:
cols: list[str] = [
    "Nuclear",
    "Biomass",
    "Waste",
    "Oil",
    "Natural Gas",
    "Hydrogen",
    "CCS Gas",
    "CCS Biomass",
    "Pumped Storage Hydroelectric",
    "Hydro",
    "Battery",
    "Compressed Air",
    "Liquid Air",
    "Wind Offshore",
    "Wind Onshore",
    "Solar Photovoltaics",
    "Interconnectors Import",
    "Unmet Load"
]

p_by_carrier = p_by_carrier[cols]

p_by_carrier.drop(
    (p_by_carrier.max()[p_by_carrier.max() < 50.0]).index,
    axis = 1,
    inplace = True
)

colors = {
    "Coal": "grey",
    "Diesel/Gas oil": "black",
    "Diesel/gas Diesel/Gas oil": "black",
    "Oil": "black",
    "Unmet Load": "black",
    "Anaerobic Digestion": "green",
    "Waste": "chocolate",
    "Sewage Sludge Digestion": "green",
    "Landfill Gas": "green",
    "Biomass (dedicated)": "green",
    "Biomass (co-firing)": "green",
    "Biomass": "green",
    "CCS Biomass": "darkgreen",
    "Interconnectors Import": "pink",
    "B6 import": "pink",
    "Sour gas": "lightcoral",
    "Natural Gas": "lightcoral",
    "CCS Gas": "lightcoral",
    "Hydrogen": "deeppink",
    "Nuclear": "orange",
    "Shoreline Wave": "aqua",
    "Tidal Barrage and Tidal Stream": "aqua",
    "Hydro": "turquoise",
    "Large Hydro": "turquoise",
    "Small Hydro": "turquoise",
    "Pumped Storage Hydroelectric": "darkturquoise",
    "Battery": "lime",
    "Compressed Air": "greenyellow",
    "Liquid Air": "lawngreen",
    "Wind Offshore": "lightskyblue",
    "Wind Onshore": "deepskyblue",
    "Solar Photovoltaics": "yellow"
}

# Start the plot
fig, ax = plt.subplots(1, 1)
fig.set_size_inches(15,10)

(p_by_carrier / 1e3).plot(
    kind="area",
    ax=ax,
    linewidth=0,
    color=[colors[col] for col in p_by_carrier.columns]
)

# Shrink current axis's height by 10% on the bottom
box = ax.get_position()
ax.set_position([
    box.x0,
    box.y0 + box.height * 0.1,
    box.width, box.height * 0.9
])

# Put a legend below current axis
ax.legend(
    loc="upper center",
    bbox_to_anchor=(0.52, -0.05),
    fancybox=True,
    shadow=True,
    ncol=5
)

# set the axis labels
ax.set_ylabel("GW")
ax.set_xlabel("")

# save figures if bool is true
if save_timestep_figures:
    plt.savefig(
        f"path/to/project/Clean_Power_Analysis/Graphs/{figure_name}_Timestep.png",
        format="png",
        bbox_inches="tight"
    )
    plt.savefig(
        f"path/to/project/Clean_Power_Analysis/Graphs/{figure_name}_Timestep.pdf",
        format="pdf",
        bbox_inches="tight"
    )